<a href="https://colab.research.google.com/github/solos001/Predict-Health-Costs-with-Regression/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [ ]:
dataset['region'].unique()

In [ ]:
dataset.sex.unique()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [ ]:
def data_proc(data):
  X=data.drop('expenses',axis=1)
  y=data.expenses

  num_attribs=X.select_dtypes(exclude="object").columns
  cat_attribs=X.select_dtypes(include="object").columns

  num_pipeline=Pipeline([('std_scaler', StandardScaler())])
  full_pipeline=ColumnTransformer([('num',num_pipeline,num_attribs),('cat',OneHotEncoder(),cat_attribs)])
  X_prepared=full_pipeline.fit_transform(X)
  

  
  X_train,X_test,y_train,y_test=train_test_split(X_prepared,y,train_size=0.8,test_size=0.2,random_state=0)
  y_test=y_test.to_numpy()
  y_train=y_train.to_numpy()
  return X_train,X_test,y_train,y_test


In [ ]:
X_train,X_test,y_train,y_test=data_proc(dataset.copy())

In [ ]:
def build_nn_model(n_hidden=4, n_neurons=100, learning_rate=0.001, input_shape=X_train.shape[1:],bn=False,dropout=False):
  model=keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=input_shape))
  if dropout:
      model.add(keras.layers.Dropout(rate=0.2))
  if bn:
      model.add(keras.layers.BatchNormalization())
  for hidden_layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,kernel_initializer="he_normal",activation="selu"))
    if bn:
      model.add(keras.layers.BatchNormalization())
    if dropout:
      model.add(keras.layers.Dropout(rate=0.2))
  model.add(keras.layers.Dense(1))
  adam_optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
  model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                optimizer=adam_optimizer,
                metrics=[tf.keras.metrics.MeanAbsoluteError()])  
  return model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
param_distribs = {
  "n_hidden": [1,2,3,4],
  "n_neurons": np.arange(1,100),
}
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_nn_model)
search_cv = RandomizedSearchCV(keras_reg, param_distribs,n_iter=5,cv=3,scoring='neg_mean_absolute_error')
search_cv.fit(X_train, y_train, epochs=400,validation_split=0.2,callbacks=[keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)])

In [ ]:
search_cv.best_params_

In [ ]:
model_1 = search_cv.best_estimator_.model
model_1.evaluate(X_test,y_test)

In [ ]:
model_1_error=model_1.evaluate(X_test,y_test)[0]

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
def build_linear(X,y):
  ridge=Ridge()
  params={'alpha':[1,5,10,30]}
  grid=GridSearchCV(estimator=ridge,param_grid=params,cv=3,scoring='neg_mean_absolute_error')
  grid.fit(X,y)
  return grid.best_estimator_

In [ ]:
model_2=build_linear(X_train,y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error
model_2_error=mean_absolute_error(y_test,model_2.predict(X_test))

In [ ]:
import xgboost

In [ ]:
def build_xgboost(X,y):
  X_train_,X_val,y_train_,y_val=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)
  xgb_reg = xgboost.XGBRegressor(random_state=0,learning_rate=0.1)
  # params={'learning_rate':[0.1,0.05,0.001],'reg_alpha':[1]}
  # grid=GridSearchCV(estimator=xgb_reg,param_grid=params,cv=3)
  # grid.fit(X_train_,y_train_,eval_set=[(X_val, y_val)], early_stopping_rounds=10,eval_metric='mse')
  xgb_reg.fit(X,y,eval_set=[(X_val, y_val)], early_stopping_rounds=10,eval_metric='mae')
  return xgb_reg

In [ ]:
model_3=build_xgboost(X_train,y_train)

In [ ]:
model_3_error=mean_absolute_error(y_test,model_3.predict(X_test))

In [ ]:
error={'Neural_Networks':round(model_1_error,2),
       'RidgeRegression':round(model_2_error,2),
       'XgboostRegressor':round(model_3_error,2)
       }

In [ ]:
error_data=pd.Series(error)

In [ ]:
error_data.keys()

In [ ]:
error_data.values

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8), constrained_layout=True)
ax.bar(error_data.keys(),error_data.values)
ax.set_xlabel('Models')
ax.set_ylabel('Mean Absolute Error')
ax.set_title('Models benchmark')

In [ ]:
model=model_1

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae= model.evaluate(X_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
